In [15]:
import numpy as np
from osgeo import ogr, gdal
import rasterio as rio
from sklearn.model_selection import train_test_split

In [13]:
def grid_vector_to_raster(shpfile_path, output_path, distance=False):

    # making the shapefile as an object.
    vec_ds = ogr.Open(shpfile_path) 
    # getting layer information of shapefile.
    layer = vec_ds.GetLayer() 
    PIXEL_SIZE = 98
    NO_DATA_VALUE = -9999
    x_min, x_max, y_min, y_max = layer.GetExtent()
    
    x_res = int((x_max - x_min) // PIXEL_SIZE)
    y_res= int((y_max - y_min) // PIXEL_SIZE)
    print(x_res, y_res)
    
    # get GeoTiff driver 
    driver = gdal.GetDriverByName('GTiff')

    # passing the filename, x and y direction resolution, no. of bands, new raster.
    new_raster = driver.Create(output_path, x_res, y_res, 1, gdal.GDT_Byte)

    # transforms between pixel raster space to projection coordinate space.
    new_raster.SetGeoTransform((x_min, PIXEL_SIZE, 0, y_max, 0, -PIXEL_SIZE))

    #get required raster band.
    band = new_raster.GetRasterBand(1)
    band.SetNoDataValue(NO_DATA_VALUE)
    band.FlushCache()

    #main conversion method
    gdal.RasterizeLayer(new_raster, [1], layer)
    
        
    

In [14]:
grid_vector_to_raster('data/Interpolaion/snowpoint.shp', 'snow_point.tif')

225 283


In [16]:
with rio.open('snow_point.tif', 'r') as file:
    data = file.read()
    

In [22]:
data.shape

(1, 283, 225)

In [26]:
train, test = train_test_split(data[0], test_size=0.2,train_size=0.8, random_state=32)

In [30]:
train.shape

(226, 225)

In [31]:
test.shape

(57, 225)